In [2]:
import os
print(os.listdir("../input"))

import pandas as pd
import numpy as np
import bs4
import re
from unicodedata import normalize
from nltk.corpus import stopwords
import operator
from keras.preprocessing import sequence
from tqdm import tqdm_notebook, tqdm

['Tripadvisor_SampleData.csv']


In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
imdb_df = pd.read_csv("../input/Tripadvisor_SampleData.csv", delimiter=',')

In [14]:
imdb_df.head()

,content,stars,title
0,"O quarto é amplo e silencioso, com uma cama co...",1,Não recomendo
1,O melhor hotel das imediações. Ótimo atendimen...,4,Satisfeito
2,localização interessanteacesso fácil ao local ...,5,HOSPÉDAGEM
3,"Localização central, quartos espaçosos e limpo...",2,Hospedagem seminário de contabilidade
4,"Poderia melhorar a qualidade do cafe da manha,...",3,JÁ FOI MELHOR


In [15]:
imdb_data = imdb_df.loc[(imdb_df['stars'] != '3')]
imdb_data = imdb_data.loc[(imdb_data['stars'] != 'stars')]
imdb_data = pd.DataFrame(imdb_data)

imdb_data.loc[(imdb_data['stars'] == '1', 'stars')] = 0
imdb_data.loc[(imdb_data['stars'] == '2', 'stars')] = 0

imdb_data.loc[(imdb_data['stars'] == '4', 'stars')] = 1
imdb_data.loc[(imdb_data['stars'] == '5', 'stars')] = 1

In [16]:
imdb_data

,content,stars,title
0,"O quarto é amplo e silencioso, com uma cama co...",0,Não recomendo
1,O melhor hotel das imediações. Ótimo atendimen...,1,Satisfeito
2,localização interessanteacesso fácil ao local ...,1,HOSPÉDAGEM
3,"Localização central, quartos espaçosos e limpo...",0,Hospedagem seminário de contabilidade
5,"Boa tarde sr. Daniel,",1,Excelente Custo-Benefício com boa localização.
6,Boa tarde!,1,Agradável estadia em POA
7,"Boa tarde,",1,Muito recomendável
8,"Olá,",1,Muito bom para viagens a negócios
10,"Por se tratar da rede Novotel, achei que fosse...",1,Ótimo atendimento
11,Recepcionistas carrancudos e secos sem o prepa...,1,Hotel moderno com excelente café


<h1>Processamento dos Comentários</h1>

In [17]:
def preprocessing_data(filepath, topwords, maxlen):
    
    imdb_data = pd.read_csv(filepath, delimiter=',')
    
    imdb_data = imdb_data.loc[(imdb_data['stars'] != '3')]
    imdb_data = imdb_data.loc[(imdb_data['stars'] != 'stars')]
    imdb_data = pd.DataFrame(imdb_data)
    
    imdb_data.loc[(imdb_data['stars'] == '1', 'stars')] = 0
    imdb_data.loc[(imdb_data['stars'] == '2', 'stars')] = 0

    imdb_data.loc[(imdb_data['stars'] == '4', 'stars')] = 1
    imdb_data.loc[(imdb_data['stars'] == '5', 'stars')] = 1

    
    reviews = []
    
    #pbar = tqdm_notebook(total=len(imdb_data))
    for review in imdb_data['content']:
        # Remover tags HTML
        review_text = bs4.BeautifulSoup(review, 'html.parser').get_text()
        
        # Remover caracteres especiais, pontuacao e numeros
        review_text = re.sub('[^a-zA-Z]', ' ', review_text)
        
        # Converter para caixa baixa
        review_text = review_text.lower()
        
        # Vetorizar o comentário
        review_words = review_text.split()
        
        # Remover stopwords
        stops = stopwords.words('portuguese')
        
        meaningful_words = [word for word in review_words if not word in stops]
        
        reviews.append(meaningful_words)
        #pbar.update(1)
    
    # Construindo dicionário de frequencia
    freq_dict = {}
    
    for review in reviews:
        for word in review:
            if not word in freq_dict:
                freq_dict[word] = 0
            freq_dict[word] += 1
    
    # Selecionar as top-K palavras (jeito inteligente Ass: carlos)
    sorted_tup = sorted(freq_dict.items(), key=operator.itemgetter(1), reverse=True)
    
    word_to_id = {}
    cnt = topwords - 1
    # Top-K palavras
    for i in sorted_tup[:topwords]:
        word_to_id[i[0]] = cnt
        cnt -= 1
    # Restante
    for i in sorted_tup[topwords:]:
        word_to_id[i[0]] = 0
    
    # Mapeando palavras para um id do dicionário
    processed_data = []
    
    for review in reviews:
        aux = []
        for word in review:
            aux.append(word_to_id[word])
        
        processed_data.append(aux)
    
    # Realizando o padding dos comentarios
    ## importar sequence de keras.preprocessing
    processed_data = np.asarray(processed_data)
    processed_data = sequence.pad_sequences(processed_data, maxlen)
    
    sentiment = imdb_data['stars'].get_values()
    
    return processed_data, sentiment, word_to_id

In [19]:
data, target, word_to_id = preprocessing_data('../input//Tripadvisor_SampleData.csv', 5000, 100)

In [20]:
data[0], target[0]

(array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0, 4991, 4785, 4687, 4965, 4971, 4990, 4665, 2768,
        2628, 4580, 3015,  639,  638, 4827, 3259, 4931, 1729, 4684, 3662,
        4694], dtype=int32), 0)

<h1>Criando o Modelo</h1>

In [21]:
from keras.models import Model
from keras.layers import *

In [22]:
input_node = Input(shape=(100,))

embedding = Embedding(input_dim=5000, 
                      input_length=100, 
                      output_dim=32)(input_node)
dropout = Dropout(0.5)(embedding)
lstm_1 = LSTM(100)(dropout)
dropout = Dropout(0.5)(lstm_1)
fc1 = Dense(1, activation='sigmoid')(dropout)

model = Model(input_node, fc1)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 32)           160000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 32)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.compile(loss='binary_crossentropy', optimizer='Adam',
              metrics=['accuracy'])

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X_train, X_test, y_train, y_test = train_test_split(data, target, 
                                                    test_size=0.33)

In [26]:
y_train, y_test

(array([1, 1, 1, ..., 1, 1, 1], dtype=object),
 array([1, 1, 1, ..., 1, 0, 1], dtype=object))

In [27]:
from keras.callbacks import *

In [28]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=1e-6, patience=3)

In [29]:
cb_list = [early_stopping]

In [30]:
model.fit(X_train, y_train, batch_size=64, epochs=20,
         validation_data=(X_test, y_test), callbacks=cb_list)

Train on 6383 samples, validate on 3144 samples
Epoch 1/20
6383/6383 [==============================] - 27s 4ms/step - loss: 0.3491 - acc: 0.9058 - val_loss: 0.3067 - val_acc: 0.9078
Epoch 2/20
6383/6383 [==============================] - 25s 4ms/step - loss: 0.2988 - acc: 0.9113 - val_loss: 0.2994 - val_acc: 0.9078
Epoch 3/20
6383/6383 [==============================] - 25s 4ms/step - loss: 0.2627 - acc: 0.9123 - val_loss: 0.3021 - val_acc: 0.9071
Epoch 4/20
6383/6383 [==============================] - 26s 4ms/step - loss: 0.2187 - acc: 0.9201 - val_loss: 0.3150 - val_acc: 0.9027
Epoch 5/20
6383/6383 [==============================] - 25s 4ms/step - loss: 0.1920 - acc: 0.9317 - val_loss: 0.3709 - val_acc: 0.9030


<h2>Testar nova entrada</h2>

In [31]:
new_review = 'Ótima localização, com vista linda e café da manhã maravilhoso!'

# Remover tags HTML
review_text = bs4.BeautifulSoup(new_review, 'html.parser').get_text()
# Remover caracteres especiais, pontuacao e numeros
review_text = re.sub('[^a-zA-Z]', ' ', review_text)
# Converter para caixa baixa
review_text = review_text.lower()
# Vetorizar o comentário
review_words = review_text.split()
# Remover stopwords
stops = stopwords.words('portuguese')

meaningful_words = [word for word in review_words if not word in stops]

processed_new_reviews = []
for word in meaningful_words:
    processed_new_reviews.append(word_to_id[word])

processed_data = np.asarray(processed_new_reviews).reshape(1, len(processed_new_reviews))
processed_data = sequence.pad_sequences(processed_data, 100)

In [32]:
processed_data

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 4962, 4984, 4927, 4788, 4997, 4995,
        4884]], dtype=int32)

In [33]:
y_pred = model.predict(processed_data)[0]

if np.round(y_pred) == 1:
    sent = 'positivo'
else:
    sent = 'negativo'

print('A predição do sentimento para a entrada \"{}\" é {}'.format(new_review, sent))

A predição do sentimento para a entrada "Ótima localização, com vista linda e café da manhã maravilhoso!" é positivo


In [ ]:
y_pred